## 3.5 Informed (Heuristic) Search Strategies

The hints come in the form of a **heuristic function**, denoted h(n).

h(n) = estimated cost of the cheapest path from the state at node n to a goal state.

### 3.5.1 Greedy best-first search

Greedy best-first search is a form of best-first search that expands first the node with the lowest h(n) value.

The solution it found does not have optimal cost, this is why the algorithm is called “greedy”—on each iteration it tries to get as close to a goal as it can, but greediness can lead to worse results than being careful. Greedy best-first graph search is complete in finite state spaces, but not in infinite ones. The worst-case time and space complexity is **O(|V|)**.

### 3.5.2 A* search

**f(n) = g(n) + h(n)**, where g(n) is the path cost from the initial state to node n, and h(n) is the estimated cost of the shortest path from n to a goal state, so we have f(n) = estimated cost of the best path that continues from n to a goal.

A* search is complete.11 Whether A* is **cost-optimal** depends on certain properties of the heuristic. A key property is **admissibility**: an admissible heuristic is one that never overestimates the cost to reach a goal. (optimistic.)

A slightly stronger property is called **consistency**. A heuristic h(n) is consistent if, for every node n and every successor n′of n generated by an action a, we have: h(n) ≤ c(n,a,n′) + h(n′). This is a form of the **triangle inequality**, which stipulates that a side of a triangle cannot be longer than the sum of the other two sides.

That A* search is complete, cost-optimal, and optimally efficient.

### 3.5.4 Satisficing search: Inadmissible heuristics and weighted A*

A* search has many good qualities, but it expands a lot of nodes. We can explore fewer
nodes if we are willing to accept solutions that are suboptimal, but are “good enough”—what we call satisficing solutions. If we allow A* search to use an **inadmissible heuristic**—one that may overestimate—.

We can apply this idea to any problem, with an approach called **weighted A*search** where we weight the heuristic value more heavily, giving us the evaluation function f(n) = g(n) + W×h(n), for some W >1.

<img src="https://iili.io/Fg6jM1s.md.png" width="250px">

In **bounded suboptimal search**, we look for a solution that is guaranteed to be within a constant factor W of the optimal cost. In **bounded-cost search**, we look for a solution whose cost is less than some constant C. And in **unbounded-cost search**, we accept a solution of any cost, as long as we can find it quickly. An example of an unbounded-cost search algorithm is **speedy search**, which is a version of greedy best-first search that uses as a heuristic the estimated number of actions required to reach a goal, regardless of the cost of those action.

### 3.5.5 Memory-bounded search

We can keep **reference counts** of the number of times a state has been reached, and remove it from the reached table.

**Beam search** limits the size of the frontier. The easiest approach is to keep only the k nodes with the best f -scores, discarding any other expanded nodes. This of course makes the search incomplete and suboptimal.

**Iterative-deepening A∗ search (IDA∗)** gives us the benefits of A∗ without the requirement to keep all reached states in memory, at a cost of visiting some states multiple times. In IDA∗ the cutoff is the f cost (g + h); at each iteration, the cutoff value is the smallest f-cost of any node that exceeded the cutoff on the previous iteration.

**Recursive best-first search (RBFS)** attempts to mimic the operation standard best-first search, but using only **linear space**. RBFS resembles a recursive depth-
first search, but rather than continuing indefinitely down the current path, it uses the f limit variable to keep track of the f-value of the best alternative path available from any ancestor of the current node. If the current node exceeds this limit, the recursion unwinds back to the alternative path. As the recursion unwinds, RBFS replaces the f-value of each node along the path with a **backed-up value**—the best f-value of its children.

RBFS is somewhat more efficient than IDA∗, but still suffers from excessive node re-generation. RBFS follows the path via Rimnicu Vilcea, “changes its mind” and tries Fagaras. These mind changes occur because every time the current best path is extended, its f-value is likely to increase—h is usually less optimistic for nodes closer to a goal. When this happens, the second-best path might become the best path, so the search has to backtrack to follow it. Each mind change corresponds to an iteration of IDA∗and could require many reexpansions of forgotten nodes to recreate the best path and extend it one more node.

IDA∗ and RBFS suffer from using too little memory. Both algorithms may end up reexploring the same states many times over.

Two algorithms that do this are **MA∗(memory-bounded A∗) and SMA∗ (simplified MA∗)**. SMA* proceeds just like A∗, expanding the best leaf until memory is full. At this point, it cannot add a new node to the search tree without dropping an old one. SMA∗always drops the worst leaf node—the one with the highest f -value. If all the leaf nodes have the same f -value, SMA∗expands the newest best leaf and deletes the
oldest worst leaf. SMA∗is complete if there is any reachable solution.

The extra time required for repeated regeneration of the same nodes means that problems that would be practically solvable by A∗, given unlimited memory, become intractable for SMA∗. Memory limitations can make a problem intractable from the point of view of computation time.

In [1]:
import math

def recursive_best_first_search(problem):
    node = Node(state=problem.initial)
    node.f = problem.heuristic(node.state)
    result, _ = rbfs(problem, node, math.inf)
    return result

def rbfs(problem, node, f_limit):
    if problem.is_goal(node.state):
        return node, 0

    successors = list(expand(problem, node))
    if not successors:
        return None, math.inf

    for s in successors:
        s.f = max(s.path_cost + problem.heuristic(s.state), getattr(node, 'f', 0))

    while True:
        successors.sort(key=lambda n: n.f)
        best = successors[0]

        if best.f > f_limit:
            return None, best.f

        alternative = successors[1].f if len(successors) > 1 else math.inf
        result, best.f = rbfs(problem, best, min(f_limit, alternative))
        if result is not None:
            return result, best.f

### 3.5.6 Bidirectional heuristic search

With unidirectional best-first search, we saw that using f (n) = g(n) + h(n) as the evaluation function gives us an A∗search that is guaranteed to find optimal-cost solutions.

With bidirectional search, it is not individual nodes but rather **pairs of nodes (one from each frontier)** that can be proved to be surely expanded.

Although both forward and backward searches are solving the same problem, they have different evaluation functions because.

The difficulty is that we don’t know for sure which node is best to expand, and therefore no bidirectional search algorithm can be guaranteed to be optimally efficient.

We have described an approach where the hF heuristic estimates the distance to the goal and hB estimates the distance to the start. This is called a **front-to-end** search. An alternative, called **front-to-front** search, attempts to estimate the distance to the other frontier. 

Bidirectional search with the f2 evaluation function and an admissible heuristic h is complete and optimal.

<img src="https://iili.io/FrGy89n.md.png" width="500px">

[breadth first](https://www.youtube.com/watch?v=1wu2sojwsyQ)

[breadth first with visited](https://www.youtube.com/watch?v=n3fPL9q_Nyc)

[depth first](https://www.youtube.com/watch?v=h1RYvCfuoN4)

[iterative deepening](https://www.youtube.com/watch?v=Y85ECk_H3h4)

[greedy best first](https://www.youtube.com/watch?v=GVvN0ikNekw)

[a*](https://www.youtube.com/watch?v=6TsL96NAZCo)